In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import re
import os
import requests
from Bio.PDB import PDBParser, MMCIFParser, PPBuilder, is_aa
from scipy.spatial.distance import cdist

In [3]:
# Load + clean + save
file = r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Junchen\atropoprobes\SiteOfLabelling\SoL_TMT_screening\06012025_4-da_SoL_25uM_on-bead_jurkat_PSMs.txt"
cleaned_file = file.replace(".txt", "_dotHeaders.txt")

df = pd.read_csv(file, sep="\t")
df.columns = df.columns.str.replace(" ", ".", regex=False)
df.to_csv(cleaned_file, sep="\t", index=False)

udata = pd.read_csv(cleaned_file, sep="\t")

train_set = pd.read_csv(
    r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Louis\SOL Processing\SoL-7-8_merged_PSMs_040722.txt",
    sep="\t"
)

human_proteome = pd.read_excel(r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Junchen\Public_data\human_proteome_2025.xlsx")
human_proteome.columns = human_proteome.columns.str.strip()

In [4]:
def extract_labeled(uData, modIDs):
    mod_regex = '|'.join(modIDs)
    return uData[uData['Modifications'].str.contains(mod_regex, na=False)].copy()

def process_psms_tmt(uData, modIDs, train_set, ptmRS=True, quant_level="MS3"):
    base_cols = [
        "Confidence", "PSM.Ambiguity", "Annotated.Sequence", "Modifications", "Master.Protein.Accessions",
        "Protein.Accessions", "Number.of.Missed.Cleavages", "Charge", "Delta.Score", "Delta.Cn",
        "mz.in.Da", "MHplus.in.Da", "Theo.MHplus.in.Da", "Delta.M.in.ppm", "Delta.mz.in.Da",
        "Isolation.Interference.in.Percent", "Average.Reporter.SN", "Ion.Inject.Time.in.ms",
        "RT.in.min", "First.Scan", "Spectrum.File", "File.ID", "Quan.Info", "XCorr"
    ]

    tmt10 = [
        "Abundance.126", "Abundance.127N", "Abundance.127C", "Abundance.128N", "Abundance.128C",
        "Abundance.129N", "Abundance.129C", "Abundance.130N", "Abundance.130C", "Abundance.131"
    ]
    base_cols += tmt10

    if quant_level == "MS3":
        base_cols.append("SPS.Mass.Matches.in.Percent")
    if ptmRS and "ptmRS.Best.Site.Probabilities" in uData.columns:
        base_cols.append("ptmRS.Best.Site.Probabilities")

    uData = uData[base_cols].copy()

    # 🧬 Clean protein accessions
    uData['Master.Protein.Accessions'] = uData['Master.Protein.Accessions'].replace("", np.nan)
    uData['Master.Protein.Accessions'] = uData['Master.Protein.Accessions'].fillna(uData['Protein.Accessions'])
    uData['Master.Protein.Accessions'] = uData['Master.Protein.Accessions'].str.split(";").str[0]
    uData["Prot_totalPSMs"] = uData.groupby("Master.Protein.Accessions")["Master.Protein.Accessions"].transform("count")

    # 🧬 Extract core sequence
    uData['Upper_Seq'] = uData['Annotated.Sequence'].str.split(r'\.', expand=True)[1].str.upper()

    # 🧼 Clean mods
    mods = uData['Modifications'].str.replace(";", "", regex=False).str.replace(" ", "", regex=False)
    mods = mods.str.replace(r'M\d*\(Oxidation\)', '', regex=True)
    mods = mods.str.replace(r'C\d*\(Carbamidomethyl\)', '', regex=True)

    mod_pattern = r'[A-Z]\d+\([^\)]+\)'
    mod_sites = mods.str.findall(mod_pattern)

    # Retain TMT mods ✅
    uData['label_sites'] = mod_sites
    uData = uData.explode("label_sites").reset_index(drop=True)

    uData["label_site"] = uData["label_sites"]
    uData["label_AA"] = uData["label_site"].str[0]
    uData["label_loc"] = uData["label_site"].str.extract(r'([0-9]+)')
    uData.drop(columns=["label_sites"], inplace=True)

    for mod in modIDs:
        uData[mod] = uData['Modifications'].str.contains(mod, na=False).astype(int)

    uData['scanID'] = uData['First.Scan'].astype(str) + "_" + uData['File.ID'].astype(str)
    uData['scanID_pep'] = uData['scanID'] + "_" + uData['Annotated.Sequence'].str.upper()
    uData['uniqueID'] = uData['scanID_pep'] + "_" + uData['Modifications']

    uData['pepLength'] = uData['Upper_Seq'].str.len()
    uData['numTMT'] = uData['Modifications'].str.count("TMT")
    uData['pepLength_numTMT'] = uData['pepLength'].astype(str) + "_" + uData['numTMT'].astype(str)

    # 🧪 Labeling
    labeled = extract_labeled(uData, modIDs)
    labeled['labeled'] = 1
    uData['labeled'] = uData['uniqueID'].isin(labeled['uniqueID']).astype(int)
    unlabeled = uData[uData['labeled'] == 0].copy()

    # 📊 Features
    uData['numPSMs'] = uData.groupby('scanID')['scanID'].transform('count')
    uData['agreePSMs'] = uData.groupby('scanID_pep')['scanID_pep'].transform('count') / uData['numPSMs']
    uData['scoreDiff'] = uData['Delta.Score'].fillna(uData['Delta.Cn'])

    rt_avg_upl = unlabeled.groupby("pepLength_numTMT")["RT.in.min"].mean().rename("RTavg_fromUPL")
    rt_avg_lpl = labeled.groupby("pepLength_numTMT")["RT.in.min"].mean().rename("RTavg_fromLPL")
    rt_means = pd.concat([rt_avg_upl, rt_avg_lpl], axis=1).reset_index()
    uData = uData.merge(rt_means, on="pepLength_numTMT", how="left")
    uData['RT_Diff_fromUPL'] = uData['RTavg_fromUPL'] - uData['RT.in.min']

    # 🚫 Drop invalids
    scale_cols = ["numPSMs", "scoreDiff", "RT_Diff_fromUPL", "agreePSMs"]
    train_set = train_set.dropna(subset=scale_cols).copy()
    uData = uData.dropna(subset=scale_cols).copy()

    # ⚙️ Normalize
    scaler = StandardScaler()
    scaler.fit(train_set[scale_cols])
    train_scaled = scaler.transform(train_set[scale_cols])
    test_scaled = scaler.transform(uData[scale_cols])

    model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
    model.fit(train_scaled, train_set['labeled'])

    probs_train = model.predict_proba(train_scaled)[:, 1]
    probs_test = model.predict_proba(test_scaled)[:, 1]
    preds_train = np.where(probs_train > 0.5, "Labeled", "Unlabeled")
    preds_test = np.where(probs_test > 0.5, "Labeled", "Unlabeled")

    # Output formatting
    train_out = train_set.copy()
    train_out["glm.probs"] = probs_train
    train_out["glm.pred"] = preds_train
    train_out["model_set"] = "train"

    test_out = uData.copy()
    test_out["glm.probs"] = probs_test
    test_out["glm.pred"] = preds_test
    test_out["model_set"] = "test"
    test_out["label_confidence"] = np.where(
        test_out["glm.probs"] >= 0.85, "High",
        np.where(test_out["glm.probs"] >= 0.5, "Medium", "Low")
    )

    return test_out

In [5]:
def clean_and_sort_sites(s):
    if not s: return []
    sites = s.split(";")
    filtered = [re.sub(r'\([^\)]+\)', '', x) for x in sites if "TMT6plex" not in x]
    freq = pd.Series(filtered).value_counts().to_dict()
    sorted_sites = sorted(set(filtered), key=lambda x: int(re.search(r'\d+', x).group()))
    return sorted_sites, freq

def process_peps_tmt(processedPSMs, modIDs, plex="10plex", human_proteome=None):
    processedPSMs = processedPSMs.copy()

    # 🔬 Sort for top PSM per scan
    processedPSMs_sorted = processedPSMs.sort_values(
        by=["scanID", "Prot_totalPSMs", "XCorr"], ascending=[True, False, False]
    )
    uniqueScans = processedPSMs_sorted.drop_duplicates("scanID").copy()

    # 🧠 Construct peptide labels
    pep_label_series = uniqueScans["Upper_Seq"].astype(str)
    for mod in modIDs:
        pep_label_series += "_" + uniqueScans[mod].astype(str)
    uniqueScans["pep_label"] = pep_label_series

    # 📋 Basic metadata
    pepDF = uniqueScans[["pep_label", "Master.Protein.Accessions"]].rename(
        columns={"pep_label": "Unique_ID", "Master.Protein.Accessions": "Uniprot_ID"}
    ).drop_duplicates()

    # 🔢 Spectra count
    spectra_count = uniqueScans["pep_label"].value_counts().rename("#_Spectra").reset_index()
    spectra_count.columns = ["Unique_ID", "#_Spectra"]
    pepDF = pepDF.merge(spectra_count, on="Unique_ID", how="left")

    # 🎯 Apply labels to all PSMs
    processedPSMs["pep_label"] = processedPSMs["Upper_Seq"].astype(str)
    for mod in modIDs:
        processedPSMs["pep_label"] += "_" + processedPSMs[mod].astype(str)

    # 💰 TMT data
    tmt10 = [
        "Abundance.126", "Abundance.127N", "Abundance.127C", "Abundance.128N", "Abundance.128C",
        "Abundance.129N", "Abundance.129C", "Abundance.130N", "Abundance.130C", "Abundance.131"
    ]
    tmt_cols = tmt10 if plex == "10plex" else []
    if tmt_cols:
        tmtData = uniqueScans[tmt_cols + ["pep_label"]].copy()
        tmtData[tmt_cols] = tmtData[tmt_cols].fillna(1)
        summedTMT = tmtData.groupby("pep_label")[tmt_cols].sum().reset_index()
        summedTMT = summedTMT.rename(columns={"pep_label": "Unique_ID"})
        pepDF = pepDF.merge(summedTMT, on="Unique_ID", how="left")

    # 📈 Averages
    features_to_avg = ["glm.probs", "XCorr", "scoreDiff", "numPSMs", "RT_Diff_fromUPL"]
    avg_feats = processedPSMs.groupby("pep_label")[features_to_avg].mean().reset_index()
    avg_feats = avg_feats.rename(columns={"pep_label": "Unique_ID"})
    pepDF = pepDF.merge(avg_feats, on="Unique_ID", how="left")

    # 🔝 Max probability
    max_glm = processedPSMs.groupby("pep_label")["glm.probs"].max().reset_index()
    max_glm.columns = ["Unique_ID", "max.glm.probs"]
    pepDF = pepDF.merge(max_glm, on="Unique_ID", how="left")

    # 🧬 Label site analysis
    label_data = processedPSMs[["pep_label", "label_site"]].dropna()
    site_lists = label_data.groupby("pep_label")["label_site"].apply(lambda x: ";".join(x)).reset_index()
    site_lists = site_lists.rename(columns={"pep_label": "Unique_ID", "label_site": "site_string"})

    def process_site_string(s):
        sorted_sites, freq_map = clean_and_sort_sites(s)
        aas = [site[0] for site in sorted_sites]
        locs = [re.search(r'\d+', site).group() for site in sorted_sites]
        site_psms = [str(freq_map[site]) for site in sorted_sites]
        return pd.Series({
            "sites": ";".join(sorted_sites),
            "aas": ";".join(aas),
            "locs": ";".join(locs),
            "site_psms": ";".join(site_psms),
            "#_PSMs": sum(map(int, site_psms))
        })

    site_details = site_lists["site_string"].apply(process_site_string)
    site_summary = pd.concat([site_lists["Unique_ID"], site_details], axis=1)
    pepDF = pepDF.merge(site_summary, on="Unique_ID", how="left")

    # 🧪 Extract core peptide sequence
    pepDF["PEPTIDE.SEQUENCE"] = pepDF["Unique_ID"].str.extract(r"^(.+?)_")

    if human_proteome is not None:
        # 🔁 Map gene and sequence
        prot_for_seq = human_proteome.rename(columns={
            "Gene Names (primary)": "Gene name",
            "Sequence": "Protein_Sequence"
        })

        pepDF = pepDF.merge(prot_for_seq[["Entry", "Gene name", "Protein_Sequence"]],
                            left_on="Uniprot_ID", right_on="Entry", how="left")

        # 🧬 New: TARG_PEPRANGE logic
        def build_peptide_range(row):
            pep = row["PEPTIDE.SEQUENCE"]
            full_seq = row["Protein_Sequence"]
            uid = row["Uniprot_ID"]
            if pd.isna(pep) or pd.isna(full_seq) or pd.isna(uid): return ""
            start = full_seq.find(pep)
            if start == -1:
                return ""
            end = start + len(pep)
            return f"{uid}_{start+1}_{end}"

        pepDF["TARG_PEPRANGE"] = pepDF.apply(build_peptide_range, axis=1)
        pepDF.drop(columns=["Entry", "Protein_Sequence"], inplace=True)

        # 🔬 Structural & functional info
        feature_cols = [
            "Protein names", "Sequence", "Active site", "Binding site", "Catalytic activity",
            "Cofactor", "DNA binding", "Site", "PDB", "Alphafold", "ChEMBL"
        ]
        missing = [col for col in feature_cols if col not in human_proteome.columns]
        if missing:
            print(f"⚠️ Missing columns: {missing}")
        else:
            pepDF = pepDF.merge(human_proteome[["Entry"] + feature_cols],
                                left_on="Uniprot_ID", right_on="Entry", how="left")
            pepDF.drop(columns=["Entry"], inplace=True)

    # 🧬 Fix Unique_ID format
    if "Gene name" in pepDF.columns:
        pepDF["Unique_ID"] = pepDF["Unique_ID"].str.replace(r"_\d+$", "", regex=True) + "_" + pepDF["Gene name"].fillna("NA")

    # 🧼 Finalize
    pepDF.fillna("", inplace=True)

    # 📦 Order columns
    cols = list(pepDF.columns)
    if "#_PSMs" in cols and "#_Spectra" in cols:
        cols.remove("#_PSMs")
        cols.insert(cols.index("#_Spectra") + 1, "#_PSMs")
    if "Gene name" in cols and "Uniprot_ID" in cols:
        cols.remove("Gene name")
        cols.insert(cols.index("Uniprot_ID") + 1, "Gene name")

    return pepDF[cols]

In [6]:
# 🧬 Define modification keyword(s) and export a cleaned PSM file

modIDs = ["DAL"]  # matches any mod column containing "DAL"

# 🔁 Filter training data
train_set_filtered = train_set[
    (train_set["paired"] == 1) | (train_set["labeled"] == 0)
]

processed_psms = process_psms_tmt(
    udata,
    modIDs,
    train_set_filtered,
    ptmRS=True,
    quant_level="MS3"
)

processed_psms_filtered = processed_psms[
    (processed_psms["labeled"] == 1) &
    (processed_psms["glm.probs"] >= 0.5)
]

processed_peps = process_peps_tmt(
    processed_psms_filtered,
    modIDs,
    plex="10plex",
    human_proteome=human_proteome
)

processed_peps.to_excel(
    r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Junchen\atropoprobes\SiteOfLabelling\SoL_TMT_screening\processed\06012025_4-da_SoL_25uM_on-bead_jurkat_PSMs_processed.xlsx",
    index=False
)

In [ ]:
# Structural proteomics, step 1: snatching PDB and Alphaphold that contains the peptide sequence.
pdb_dir = r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Junchen\Public_data\pdb_cache"
os.makedirs(pdb_dir, exist_ok=True)

# 🧬 Initialize parsers
pdb_parser = PDBParser(QUIET=True)
cif_parser = MMCIFParser(QUIET=True)

def extract_chain_sequences(structure):
    ppb = PPBuilder()
    chain_seqs = {}
    for model in structure:
        for chain in model:
            peptides = ppb.build_peptides(chain)
            seq = ''.join(str(p.get_sequence()) for p in peptides)
            if seq:
                chain_seqs[chain.id] = seq
    return chain_seqs

def parse_structure(file_path):
    try:
        if file_path.endswith(".pdb"):
            return pdb_parser.get_structure("pdb", file_path)
        elif file_path.endswith(".cif"):
            return cif_parser.get_structure("cif", file_path)
    except Exception as e:
        print(f"❌ Parsing error for {file_path}: {e}")
    return None

def format_alphafold_id(raw_id):
    """Clean and format raw AlphaFold ID (e.g., 'O75935;') to 'AF-O75935-F1-model_v4'"""
    return f"AF-{str(raw_id).strip().replace(';', '')}-F1-model_v4"

def download_pdb_or_cif_or_alphafold(pdb_id, pdb_dir, alphafold_id=None):
    pdb_id = pdb_id.lower().strip()
    pdb_path = os.path.join(pdb_dir, f"{pdb_id}.pdb")
    cif_path = os.path.join(pdb_dir, f"{pdb_id}.cif")

    af_path = None
    full_af_id = None
    if alphafold_id:
        full_af_id = format_alphafold_id(alphafold_id)
        af_path = os.path.join(pdb_dir, f"{full_af_id}.pdb")

    if os.path.exists(pdb_path): return pdb_path
    if os.path.exists(cif_path): return cif_path
    if af_path and os.path.exists(af_path): return af_path

    try:
        r = requests.get(f"https://files.rcsb.org/download/{pdb_id.upper()}.pdb", timeout=10)
        if r.ok and "ATOM" in r.text:
            with open(pdb_path, "w") as f:
                f.write(r.text)
            return pdb_path
    except: pass

    try:
        r = requests.get(f"https://files.rcsb.org/download/{pdb_id.upper()}.cif", timeout=10)
        if r.ok:
            with open(cif_path, "w") as f:
                f.write(r.text)
            return cif_path
    except: pass

    if full_af_id:
        try:
            url = f"https://alphafold.ebi.ac.uk/files/{full_af_id}.pdb"
            r = requests.get(url, timeout=20)
            if r.ok:
                with open(af_path, "wb") as f:
                    f.write(r.content)
                return af_path
        except Exception as e:
            print(f"❌ AlphaFold download failed for {full_af_id}: {e}")

    return None

def get_best_pdb_mapping(df, pdb_dir):
    df = df.copy()
    df["PDB_clean"] = ""

    i = 0
    while i < len(df):
        row = df.iloc[i]
        pdb_string = str(row.get("PDB", ""))
        uniprot_id = row["Uniprot_ID"]
        alphafold_id = row.get("Alphafold", "")

        if len(pdb_string) > 4 and ";" in pdb_string:
            pep_seqs = df[df["Uniprot_ID"] == uniprot_id]["PEPTIDE.SEQUENCE"].dropna().unique().tolist()
            pdbs_to_check = pdb_string.split(";")
            match_counts = []

            for pdb_id in pdbs_to_check:
                structure_file = download_pdb_or_cif_or_alphafold(pdb_id, pdb_dir, alphafold_id)
                if not structure_file:
                    match_counts.append(0)
                    continue

                structure = parse_structure(structure_file)
                if not structure:
                    match_counts.append(0)
                    continue

                chain_seqs = extract_chain_sequences(structure).values()
                match_count = sum(any(pep in chain_seq for chain_seq in chain_seqs) for pep in pep_seqs)
                match_counts.append(match_count)

            if match_counts:
                max_count = max(match_counts)
                best_pdbs = [pdbs_to_check[j] for j, count in enumerate(match_counts) if count == max_count]
                df.loc[df["Uniprot_ID"] == uniprot_id, "PDB_clean"] = ";".join(best_pdbs)

            i += len(pep_seqs)

        else:
            if not pdb_string or pdb_string in ["0", "nan", "None"]:
                if pd.notna(alphafold_id) and alphafold_id.strip():
                    df.at[i, "PDB_clean"] = format_alphafold_id(alphafold_id)
                else:
                    df.at[i, "PDB_clean"] = ""
            else:
                df.at[i, "PDB_clean"] = pdb_string
            i += 1

    return df

# 🧬 Input any processed file
excel_path = r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Junchen\atropoprobes\SiteOfLabelling\SoL_TMT_screening\processed\05152025_6-da_SoL_25uM_on-bead_jurkat_sup_fasta_PSMs_processed.xlsx"
sol = pd.read_excel(excel_path)

# 🚀 Run mapping
sol_with_pdb_clean = get_best_pdb_mapping(sol, pdb_dir)

# 💾 Save output
sol_with_pdb_clean.to_csv(
    r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Junchen\atropoprobes\SiteOfLabelling\SoL_TMT_screening\processed\05152025_6-da_SoL_25uM_on-bead_jurkat_sup_fasta_PSMs_w_PDB_filter.csv",
    index=False
)

In [25]:
# Structural proteomics, step 2: calculating the distance between the labeled peptides and "active sites" using PDB, or Alphafold

# === 📁 SETUP ===
def format_alphafold_filename(af_id):
    """Ensure AlphaFold ID is formatted like AF-<UniProt>-F1-model_v4"""
    if pd.isna(af_id) or not af_id.strip(): return ""
    af_id = af_id.strip().split(";")[0]
    if not af_id.startswith("AF-"):
        af_id = f"AF-{af_id}-F1-model_v4"
    return af_id

def fetch_structure(pdb_id, cache_dir="pdb_cache", alphafold_id=None):
    os.makedirs(cache_dir, exist_ok=True)
    pdb_id = pdb_id.lower().strip()
    alphafold_id = format_alphafold_filename(alphafold_id)

    pdb_file = os.path.join(cache_dir, f"{pdb_id}.pdb")
    cif_file = os.path.join(cache_dir, f"{pdb_id}.cif")
    af_file = os.path.join(cache_dir, f"{alphafold_id}.pdb") if alphafold_id else None

    if not os.path.exists(pdb_file):
        r = requests.get(f"https://files.rcsb.org/download/{pdb_id.upper()}.pdb")
        if r.ok and "ATOM" in r.text:
            with open(pdb_file, "w") as f:
                f.write(r.text)

    if not os.path.exists(pdb_file) and not os.path.exists(cif_file):
        r = requests.get(f"https://files.rcsb.org/download/{pdb_id.upper()}.cif")
        if r.ok:
            with open(cif_file, "w") as f:
                f.write(r.text)

    if not os.path.exists(pdb_file) and not os.path.exists(cif_file) and alphafold_id:
        url = f"https://alphafold.ebi.ac.uk/files/{alphafold_id}.pdb"
        try:
            r = requests.get(url, timeout=20)
            if r.ok:
                with open(af_file, "wb") as f:
                    f.write(r.content)
        except Exception as e:
            print(f"❌ AlphaFold download failed for {alphafold_id}: {e}")

    if os.path.exists(pdb_file): return pdb_file
    if os.path.exists(cif_file): return cif_file
    if alphafold_id and os.path.exists(af_file): return af_file
    return None

# === PDB parsing functions remain unchanged ===
def parse_structure(path):
    if path.endswith(".pdb"):
        return PDBParser(QUIET=True).get_structure("pdb", path)
    elif path.endswith(".cif"):
        return MMCIFParser(QUIET=True).get_structure("cif", path)
    return None

def extract_chain_residues(structure):
    chain_map = {}
    for model in structure:
        for chain in model:
            coords, resnos = [], []
            for res in chain:
                if is_aa(res, standard=True):
                    try:
                        coords.append(res["CA"].coord)
                        resnos.append(res.id[1])
                    except KeyError:
                        continue
            if coords:
                chain_map[chain.id] = {"coords": np.array(coords), "resnos": resnos}
    return chain_map

def extract_active_sites(active_str):
    if pd.isna(active_str): return []
    matches = re.findall(r"ACTIVE\s*(\d+)", active_str)
    return [int(m) for m in matches]

def find_min_distance(coords1, coords2):
    if not coords1 or not coords2: return np.inf
    return np.min(cdist(coords1, coords2))

def measure_active_site_distance(row, cache_dir="pdb_cache"):
    pdb_id = str(row.get("PDB_clean", ""))[:4]
    alphafold_id = row.get("AlphaFold", "")
    peptide = row.get("PEPTIDE.SEQUENCE", "")
    targ_range = row.get("TARG_PEPRANGE", "")
    ref_seq = str(row.get("Sequence", ""))
    active_str = row.get("Active site", "")

    if not pdb_id or pdb_id == "0" or pdb_id.lower() in ["nan", "none"]:
        pdb_id = alphafold_id

    if any(pd.isna(x) or x == "" for x in [pdb_id, peptide, targ_range, ref_seq, active_str]):
        return pd.Series(["", "", "", "", "Missing Info"])

    start_pos = int(targ_range.split("_")[1])
    active_sites = extract_active_sites(active_str)
    site_total = len(active_sites)

    structure_file = fetch_structure(pdb_id, cache_dir, alphafold_id=alphafold_id)
    if not structure_file:
        return pd.Series(["", "", site_total, 0, "Structure not found"])

    structure = parse_structure(structure_file)
    chain_data = extract_chain_residues(structure)

    min_dist = np.inf
    max_shift = 0
    site_in_pdb = 0
    site_near = 0

    for data in chain_data.values():
        coords, resnos = data["coords"], data["resnos"]
        try:
            pep_start_idx = resnos.index(start_pos)
        except ValueError:
            continue

        shift = resnos[pep_start_idx] - start_pos
        max_shift = max(max_shift, abs(shift))
        pep_indices = range(pep_start_idx, pep_start_idx + len(peptide))
        pep_coords = [coords[i] for i in pep_indices if i < len(coords)]

        site_coords = []
        for site in active_sites:
            try:
                idx = resnos.index(site + shift)
                site_coords.append(coords[idx])
            except ValueError:
                continue

        if site_coords:
            dist = find_min_distance(pep_coords, site_coords)
            min_dist = min(min_dist, dist)
            site_in_pdb += len(site_coords)
            site_near += sum(1 for s in site_coords if find_min_distance(pep_coords, [s]) <= 6)

    if min_dist < np.inf:
        return pd.Series([round(min_dist, 3), max_shift, site_total, site_in_pdb, site_near])
    else:
        return pd.Series(["Not Found", max_shift, site_total, site_in_pdb, site_near])

def compute_active_site_distances(sol_df, cache_dir="pdb_cache"):
    out = sol_df.copy()
    cols = ["spatial_distance", "shift", "site_total", "site_in_pdb", "site_near_peptide"]
    out[cols] = sol_df.apply(lambda row: measure_active_site_distance(row, cache_dir), axis=1)
    return out

# === 📥 LOAD AND PROCESS ===
excel_path = r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Junchen\atropoprobes\SiteOfLabelling\SoL_TMT_screening\processed\04152025_2-da_SoL_25uM_2-injections_PSMs_w_PDB_filter.csv"
sol_with_pdb_clean = pd.read_csv(excel_path)

# === 🧪 COMPUTE DISTANCES ===
sol_with_active = compute_active_site_distances(sol_with_pdb_clean, cache_dir="pdb_cache")
sol_with_active.to_csv(
    r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Junchen\atropoprobes\SiteOfLabelling\SoL_TMT_screening\processed\04152025_2-da_SoL_25uM_2-injections_PSMs_w_active_site_distances.csv",
    index=False
)


In [26]:
# Structural proteomics, step 3: calculating the distance between the labeled peptides and "binding sites" using PDB, or Alphafold
# === 📁 SETUP ===
def format_alphafold_filename(af_id):
    """Ensure AlphaFold ID is formatted like AF-<UniProt>-F1-model_v4"""
    if pd.isna(af_id) or not af_id.strip(): return ""
    af_id = af_id.strip().split(";")[0]  # remove trailing semicolon
    if not af_id.startswith("AF-"):
        af_id = f"AF-{af_id}-F1-model_v4"
    return af_id

def fetch_structure(pdb_id, cache_dir="pdb_cache", alphafold_id=None):
    os.makedirs(cache_dir, exist_ok=True)
    pdb_id = pdb_id.lower().strip()
    alphafold_id = format_alphafold_filename(alphafold_id)

    pdb_file = os.path.join(cache_dir, f"{pdb_id}.pdb")
    cif_file = os.path.join(cache_dir, f"{pdb_id}.cif")
    af_file = os.path.join(cache_dir, f"{alphafold_id}.pdb") if alphafold_id else None

    if not os.path.exists(pdb_file):
        try:
            r = requests.get(f"https://files.rcsb.org/download/{pdb_id.upper()}.pdb", timeout=10)
            if r.ok and "ATOM" in r.text:
                with open(pdb_file, "w") as f:
                    f.write(r.text)
        except: pass

    if not os.path.exists(pdb_file) and not os.path.exists(cif_file):
        try:
            r = requests.get(f"https://files.rcsb.org/download/{pdb_id.upper()}.cif", timeout=10)
            if r.ok:
                with open(cif_file, "w") as f:
                    f.write(r.text)
        except: pass

    if not os.path.exists(pdb_file) and not os.path.exists(cif_file) and alphafold_id:
        try:
            url = f"https://alphafold.ebi.ac.uk/files/{alphafold_id}.pdb"
            r = requests.get(url, timeout=20)
            if r.ok:
                with open(af_file, "wb") as f:
                    f.write(r.content)
        except Exception as e:
            print(f"❌ AlphaFold download failed for {alphafold_id}: {e}")

    if os.path.exists(pdb_file): return pdb_file
    if os.path.exists(cif_file): return cif_file
    if alphafold_id and os.path.exists(af_file): return af_file
    return None

def parse_structure(path):
    if path.endswith(".pdb"):
        return PDBParser(QUIET=True).get_structure("pdb", path)
    elif path.endswith(".cif"):
        return MMCIFParser(QUIET=True).get_structure("cif", path)
    return None

# === 🧬 DATA EXTRACTORS ===
def extract_chain_residues(structure):
    chain_map = {}
    for model in structure:
        for chain in model:
            coords, resnos = [], []
            for res in chain:
                if is_aa(res, standard=True):
                    try:
                        coords.append(res["CA"].coord)
                        resnos.append(res.id[1])
                    except KeyError:
                        continue
            if coords:
                chain_map[chain.id] = {"coords": np.array(coords), "resnos": resnos}
    return chain_map

def extract_binding_sites(binding_str):
    if pd.isna(binding_str): return []
    matches = re.findall(r"BINDING\s*(\d+)", binding_str)
    return [int(m) for m in matches]

# === ⚙️ DISTANCE COMPUTATION ===
def find_min_distance(coords1, coords2):
    if not coords1 or not coords2: return np.inf
    return np.min(cdist(coords1, coords2))

def measure_binding_site_distance(row, cache_dir="pdb_cache"):
    pdb_id = str(row.get("PDB_clean", ""))[:4]
    alphafold_id = row.get("AlphaFold", "")
    peptide = row.get("PEPTIDE.SEQUENCE", "")
    targ_range = row.get("TARG_PEPRANGE", "")
    ref_seq = str(row.get("Sequence", ""))
    binding_str = row.get("Binding site", "")

    if not pdb_id or pdb_id == "0" or pdb_id.lower() in ["nan", "none"]:
        pdb_id = alphafold_id

    if any(pd.isna(x) or x == "" for x in [pdb_id, peptide, targ_range, ref_seq, binding_str]):
        return pd.Series(["", "", "", "", "Missing Info"])

    try:
        start_pos = int(targ_range.split("_")[1])
    except:
        return pd.Series(["", "", "", "", "Bad TARG_PEPRANGE"])

    binding_sites = extract_binding_sites(binding_str)
    site_total = len(binding_sites)

    structure_file = fetch_structure(pdb_id, cache_dir, alphafold_id=alphafold_id)
    if not structure_file:
        return pd.Series(["", "", site_total, 0, "Structure not found"])

    structure = parse_structure(structure_file)
    chain_data = extract_chain_residues(structure)

    min_dist = np.inf
    max_shift = 0
    site_in_pdb = 0
    site_near = 0

    for data in chain_data.values():
        coords, resnos = data["coords"], data["resnos"]
        try:
            pep_start_idx = resnos.index(start_pos)
        except ValueError:
            continue

        shift = resnos[pep_start_idx] - start_pos
        max_shift = max(max_shift, abs(shift))
        pep_indices = range(pep_start_idx, pep_start_idx + len(peptide))
        pep_coords = [coords[i] for i in pep_indices if i < len(coords)]

        site_coords = []
        for site in binding_sites:
            try:
                idx = resnos.index(site + shift)
                site_coords.append(coords[idx])
            except ValueError:
                continue

        if site_coords:
            dist = find_min_distance(pep_coords, site_coords)
            min_dist = min(min_dist, dist)
            site_in_pdb += len(site_coords)
            site_near += sum(1 for s in site_coords if find_min_distance(pep_coords, [s]) <= 6)

    if min_dist < np.inf:
        return pd.Series([round(min_dist, 3), max_shift, site_total, site_in_pdb, site_near])
    else:
        return pd.Series(["Not Found", max_shift, site_total, site_in_pdb, site_near])

# === 📊 APPLY ACROSS PEPTIDES ===
def compute_binding_site_distances(sol_df, cache_dir="pdb_cache"):
    out = sol_df.copy()
    cols = ["spatial_distance", "shift", "site_total", "site_in_pdb", "site_near_peptide"]
    out[cols] = sol_df.apply(lambda row: measure_binding_site_distance(row, cache_dir), axis=1)
    return out

# === 📥 LOAD CSV AND PROCESS ===
csv_path = r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Junchen\atropoprobes\SiteOfLabelling\SoL_TMT_screening\processed\04152025_2-da_SoL_25uM_2-injections_PSMs_w_PDB_filter.csv"
sol_with_pdb_clean = pd.read_csv(csv_path)

# === 🧪 COMPUTE AND SAVE RESULTS ===
sol_with_binding = compute_binding_site_distances(sol_with_pdb_clean, cache_dir="pdb_cache")
sol_with_binding.to_csv(
    r"C:\Users\merce\Scripps Research Dropbox\Parker Lab\Junchen\atropoprobes\SiteOfLabelling\SoL_TMT_screening\processed\04152025_2-da_SoL_25uM_2-injections_PSMs_w_binding_site_distances.csv",
    index=False
)
